**Import modules et données puis préparation**

In [1]:
import math
from random import randrange
import numpy as np
import pandas as pd
df = pd.read_csv("info_communes_parc.csv", sep=";")

In [32]:
elite = 0.2
gueux = 0.05
mutations = 0.1
taille_echantillon = 5000
scores = np.zeros(shape=(taille_echantillon,6))
nb_etapes = 10

In [3]:
def distance (depart, arrivee) :
    tmp = df.loc[df["insee"].isin([depart,arrivee]), "lat_centre":"long_centre"]
    x_dep = tmp.iloc[0,1]
    y_dep = tmp.iloc[0,0]
    x_arr = tmp.iloc[1,1]
    y_arr = tmp.iloc[1,0]
    distance = math.acos(math.sin(math.radians(x_dep))*math.sin(math.radians(x_arr))+math.cos(math.radians(x_dep))*math.cos(math.radians(x_arr))*math.cos(math.radians(y_dep-y_arr)))*6371
    return round(distance,1)
noms_villes = df["insee"].tolist()

In [33]:
def generateur_aleatoire (n_etapes, n_iterations, liste) :
    tmp_array = np.zeros(shape=(n_iterations,n_etapes))
    for h in range(n_iterations) :
        liste_voyage = []
        tmp_liste = liste.copy()
        for i in range(n_etapes) :
            nombre_alea = randrange(0, len(tmp_liste), 1)
            liste_voyage.append(tmp_liste[nombre_alea])
            tmp_liste.pop(nombre_alea)
        tmp_array[h] =  liste_voyage
        print("ligne " + str(h),end='\r')
    return tmp_array

In [34]:
generation = generateur_aleatoire(nb_etapes,taille_echantillon, noms_villes)

In [ ]:
#on peut sauvegarder la matrice pour les futurs devs
with open("matrice.npy", "wb") as f:
    np.save(f, matrice)

**Préparation des fonctions et paramètres pour les nouvelles générations**

In [30]:
with open("matrice.npy", "rb") as f:
    matrice = np.load(f)

In [38]:
def notation_init (matrice) :
    #pense bete tri colonnes array[array[:, 1].argsort()]
    #decompte distance
    for i in range(scores.shape[0]) :
        distance_totale = 0
        for j in range(9) :
            print("nb nm " + str(i),end='\r')
            distance_totale += distance(matrice[i,j],matrice[i,j+1])
        scores[i,0] = distance_totale
    #decompte POI
    for i in range(scores.shape[0]) :
        nb_poi = 0
        for j in range(10) :
            valeur = int(df.loc[df["insee"] == matrice[i,j], "nb_poi"])
            print("nb poi " + str(i),end='\r')
            nb_poi += valeur
        scores[i,2] = nb_poi

In [39]:
notation_init(generation)

In [41]:
scores[5]

array([354.3,   0. ,  23. ,   0. ,   0. ,   0. ])

In [2]:
df = pd.read_csv("effectifs.csv",sep=";", low_memory=False)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4057201 entries, 0 to 4057200
Data columns (total 16 columns):
 #   Column              Dtype  
---  ------              -----  
 0   annee               float64
 1   patho_niv1          object 
 2   patho_niv2          object 
 3   patho_niv3          object 
 4   top                 object 
 5   cla_age_5           object 
 6   sexe                float64
 7   region              int64  
 8   dept                object 
 9   ntop                float64
 10  npop                float64
 11  prev                float64
 12  niveau_prioritaire  object 
 13  libelle_classe_age  object 
 14  libelle_sexe        object 
 15  tri                 float64
dtypes: float64(6), int64(1), object(9)
memory usage: 495.3+ MB


In [5]:
tmpliste = df["patho_niv2"].unique()

In [6]:
liste = []
for i in tmpliste :
    try :
        if ('cancer' in i)  or ('Cancer' in i) or ('cancers' in i) or ('Cancers' in i):
            liste.append(i)
    except :
        print(i)

nan


In [7]:
liste

['Autres cancers',
 'Cancer colorectal',
 'Cancer du sein de la femme',
 'Cancer du poumon',
 'Cancer de la prostate']

In [8]:
df_filtered = df[df["patho_niv2"].isin(liste)]

In [9]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 687960 entries, 56649 to 4017612
Data columns (total 16 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   annee               687960 non-null  float64
 1   patho_niv1          687960 non-null  object 
 2   patho_niv2          687960 non-null  object 
 3   patho_niv3          458640 non-null  object 
 4   top                 687960 non-null  object 
 5   cla_age_5           687960 non-null  object 
 6   sexe                687960 non-null  float64
 7   region              687960 non-null  int64  
 8   dept                687960 non-null  object 
 9   ntop                438078 non-null  float64
 10  npop                687960 non-null  float64
 11  prev                438078 non-null  float64
 12  niveau_prioritaire  687960 non-null  object 
 13  libelle_classe_age  687960 non-null  object 
 14  libelle_sexe        687960 non-null  object 
 15  tri                 687960 no

In [12]:
df_agg = df_filtered.groupby(["annee","region","libelle_classe_age"]).agg({"ntop":"sum","npop":"sum"})

In [13]:
df_agg.to_csv("export.csv")